In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'data/split/train'
validation_dir = 'data/split/val'
test_dir = 'data/split/test'

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.15, 
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, 
                                   horizontal_flip=True, fill_mode="nearest")

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64),
                                                    batch_size=32, class_mode='categorical')

validation_generator = val_test_datagen.flow_from_directory(validation_dir, target_size=(64, 64),
                                                     batch_size=32, class_mode='categorical')

test_generator = val_test_datagen.flow_from_directory(test_dir, target_size=(64, 64),
                                                      batch_size=32, class_mode='categorical')

Found 62014 images belonging to 29 classes.
Found 13284 images belonging to 29 classes.
Found 13309 images belonging to 29 classes.


In [7]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# Load the trained VGG16 model
model = load_model('asl_vgg16_model.h5')  # Update with your model path

# Define a function to preprocess a video frame for VGG16
def preprocess_frame(frame, target_size=(64, 64)):
    # Resize frame to the target size
    frame_resized = cv2.resize(frame, target_size)
    # Convert to array and normalize the image data
    frame_array = img_to_array(frame_resized) / 255.0
    # Expand dimensions to fit the model input (batch size, height, width, channels)
    frame_array = np.expand_dims(frame_array, axis=0)
    return frame_array

# Define a function to map prediction index to class label (e.g., A, B, C, etc.)
class_labels = list(train_generator.class_indices.keys())  # Assuming train_generator is available

def predict_frame(model, frame):
    # Preprocess the frame
    processed_frame = preprocess_frame(frame)
    # Predict the class
    predictions = model.predict(processed_frame)
    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]
    return predicted_class

# Open a video capture object for the webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Loop to continuously get frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image")
        break
    
    # Predict the class for the current frame
    predicted_class = predict_frame(model, frame)
    
    # Display the predicted class on the frame
    cv2.putText(frame, f'Prediction: {predicted_class}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Show the frame with the prediction
    cv2.imshow('Real-Time Prediction', frame)
    
    # Exit when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the display window
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━

: 